## Standup

0) Setup/Config problems?<br>
1) Caught up on Treehouse?<br>
2) What's your project idea?<br>
3) How are you feeling about this class?<br>
4) Tech Events?<br><br><hr>

## SQL in SQL Lite

Open an Anaconda Prmompt.
Navigate to the folder where you unpacked 'sqlintro.zip'.
Then run sql lite and tell it to open (or create) a database named 'courses':
> %> sqlite3 courses

You should see a sql lite prompt:
> sqlite>

Next we'll run an sql script I've provided to create and populate some tables. The SQL scrips uses some SQL that hasn't been covered by the treehouse videos yet. 
> sqlite> .read create_courses.sql

Here's the contents of the script: 
```SQL
DROP TABLE course;
DROP TABLE student;
DROP TABLE assignment;


CREATE TABLE course (
    id          INT         NOT NULL PRIMARY KEY,
    course_name VARCHAR(50) NOT NULL
);

CREATE TABLE student (
    id          INT         NOT NULL PRIMARY KEY,
    last_name   VARCHAR(20) NOT NULL,
    first_name  VARCHAR(20) NOT NULL
);

CREATE TABLE assignment (
    id              INT         NOT NULL PRIMARY KEY,
    assignment_name VARCHAR(20) NOT NULL,
    due_date        DATETIME    NOT NULL,
    submit_date     DATETIME    NOT NULL,
    grade           DECIMAL     NOT NULL,
    course_id       INT         NOT NULL,
    student_id      INT         NOT NULL
);

/* 
example of a multi line 
SQL comment 
*/
-- single line SQL comment  

INSERT INTO course (id,course_name) 
VALUES (1,'Math'),
       (2,'Science'),
       (3,'Social Studies'),
	   (4,'English');

INSERT INTO student (id, last_name,first_name)
VALUES (1,'Joe','Bobby'),
	   (2,'Smith','Joe'),
	   (3,'Winkle','Perry'),
	   (4,'Legend','Iam'),
	   (5,'Mary','Poppins'),
	   (6,'Bell','Tinker');

INSERT INTO assignment (id,assignment_name,due_date,submit_date,grade,course_id,student_id)
VALUES (1,'math 1','2019-06-04','2091-07-04',40,1,1),
	   (2,'math 2','2019-06-04','2019-06-15',50,1,2),
	   (3,'science 1','2019-06-04','2019-06-03',60,2,3),
	   (4,'science 2','2019-06-04','2019-06-03',70,2,4),
	   (5,'social studies 1','2019-06-04','2019-06-19',80,3,5),
	   (6,'social studies 2','2019-06-04','2019-06-13',90,3,6),
	   (7,'english 1','2019-06-04','2019-06-09',100,4,3),
	   (8,'english 2','2019-06-04','2019-07-24',85,4,6);
```

This script works for sql-lite. There is another script 'create_courses_mssql.sql' which works better for MSSQL. (We'll look at MSSQL later, time permitting.) 

Our first select statment. Note the ';' at the end. SQL lite requires this to know when you are done entering your SQL statement. (Consider multi line statements.)
> sqlite> SELECT * FROM student;

Generates the output:

> 1|Joe|Bobby<br/>
> 2|Smith|Joe<br/>
> 3|Winkle|Perry<br/>
> 4|Legend|Iam<br/>
> 5|Mary|Poppins<br/>
> 6|Bell|Tinker<br/>


Ctrl-Z then Enter to exit.
  

## SQL in python / jupyter notebooks

In [1]:
import sqlite3 as sql
import pandas as pd

# Create the connection to the sql lite database we created above
db_connection= sql.connect(r'courses')

In [3]:
# Create the dataframe from a query
df = pd.read_sql_query("SELECT * FROM assignment", db_connection)
df

,id,assignment_name,due_date,submit_date,grade,course_id,student_id
0,1,math 1,2019-06-04,2091-07-04,40,1,1
1,2,math 2,2019-06-04,2019-06-15,50,1,2
2,3,science 1,2019-06-04,2019-06-03,60,2,3
3,4,science 2,2019-06-04,2019-06-03,70,2,4
4,5,social studies 1,2019-06-04,2019-06-19,80,3,5
5,6,social studies 2,2019-06-04,2019-06-13,90,3,6
6,7,english 1,2019-06-04,2019-06-09,100,4,3
7,8,english 2,2019-06-04,2019-07-24,85,4,6


### Multiple Column Select (avoid the evil '*')

In [23]:
# A multi column query that doesn't use the evil '*'
df_multicol = pd.read_sql_query("SELECT assignment_name, due_date FROM assignment", db_connection)
df_multicol

,assignment_name,due_date
0,math 1,2019-06-04
1,math 2,2019-06-04
2,science 1,2019-06-04
3,science 2,2019-06-04
4,social studies 1,2019-06-04
5,social studies 2,2019-06-04
6,english 1,2019-06-04
7,english 2,2019-06-04


### Column Aliasing (pretty column headers)

In [5]:
# A multi column query that doesn't use the evil '*' but does use aliasing with AS
# Note that you should put quotes around aliases with spaces in them.
df_multicol = pd.read_sql_query("SELECT assignment_name AS Assignment, due_date AS 'Due Date', submit_date As 'Submitted On' FROM assignment", db_connection)
df_multicol

,Assignment,Due Date,Submitted On
0,math 1,2019-06-04,2091-07-04
1,math 2,2019-06-04,2019-06-15
2,science 1,2019-06-04,2019-06-03
3,science 2,2019-06-04,2019-06-03
4,social studies 1,2019-06-04,2019-06-19
5,social studies 2,2019-06-04,2019-06-13
6,english 1,2019-06-04,2019-06-09
7,english 2,2019-06-04,2019-07-24


### Filtering with WHERE 

In [10]:
# A query with a simple where clause that uses columns 
# select all assignments that were submitte before or on the due date
# NOTE use of """  """ to allow us to format the SQL for readability
df_filtered = pd.read_sql_query("""
SELECT 
    assignment_name AS Assignment, 
    due_date AS 'Due On', 
    submit_date As 'Submitted On' 
FROM assignment 
WHERE submit_date <= due_date
""", db_connection)
df_filtered

,Assignment,Due On,Submitted On
0,science 1,2019-06-04,2019-06-03
1,science 2,2019-06-04,2019-06-03


#### Or and And .. please use parentheses

In [16]:
# Select all the assigments with that were late and either are't course id 3 or have a D grade (between 60 and 70)
df_confusing_filter = pd.read_sql_query("""
SELECT 
        assignment_name AS Assignment, 
        due_date AS 'Due On', 
        submit_date As 'Submitted On', 
        grade AS Grade,
        course_id AS Course
    FROM assignment 
    WHERE 
        submit_date > due_date
        AND  
        (   
            grade BETWEEN 60 AND 70
            OR 
            course_id != 3
        )
""", db_connection)

df_confusing_filter

,Assignment,Due On,Submitted On,Grade,Course
0,math 1,2019-06-04,2091-07-04,40,1
1,math 2,2019-06-04,2019-06-15,50,1
2,english 1,2019-06-04,2019-06-09,100,4
3,english 2,2019-06-04,2019-07-24,85,4


OOPS! AND has precendence over OR so we need to add parentheses to make this work properly.

Note that between is inlcusive. Both start and end are included. 

#### Filtering with a list using IN and with wildcards % and _

In [60]:
df_IN = pd.read_sql_query("""
SELECT 
        assignment_name AS Assignment, 
        due_date AS 'Due On', 
        submit_date As 'Submitted On', 
        grade AS Grade,
        course_id AS Course
    FROM assignment 
    WHERE 
        (grade IN (60, 80, 85, 100, 40, 20, 0) AND grade NOT IN (20, 10) )
        OR assignment_name LIKE "%c%"     -- for most SQL LIKE is case insensitive but that can be changed!
        OR assignment_name LIKE "math _"
""", db_connection)

df_IN

,Assignment,Due On,Submitted On,Grade,Course
0,math 1,2019-06-04,2091-07-04,40,1
1,math 2,2019-06-04,2019-06-15,50,1
2,science 1,2019-06-04,2019-06-03,60,2
3,science 2,2019-06-04,2019-06-04,70,2
4,social studies 1,2019-06-04,2019-06-19,80,3
5,social studies 2,2019-06-04,2019-06-13,90,3
6,english 1,2019-06-04,2019-06-09,100,4
7,english 2,2019-06-04,2019-07-24,85,4


##### Empty values (NULL)   
In SQL NULL is not Zero (and vice versa).
You cant use = and != with NULL. You must use IS NULL and IS NOT NULL.


## Good coding practices in SQL

Avoid using Select *, specify column names intead. For several reasons:
* more readable
* performance costs with *
* fragility with *

Use parentheses for clairity.

Best practice capitalization is to put SQL keywords in ALL CAPS. Things like table names, column names and the like in all lower case. Obviously aliases should be capitialized or not as needed. 

Which is better 
```
SELECT 
        assignment_name AS Assignment, 
        due_date AS 'Due On', 
        submit_date As 'Submitted On', 
        grade AS Grade,
        course_id AS Course
    FROM assignment 
    WHERE 
        submit_date > due_date
        AND  grade BETWEEN 60 AND 70
        OR course_id != 3
```
or 
```
SELECT assignment_name AS Assignment, due_date AS 'Due On', submit_date As 'Submitted On', grade AS Grade, course_id AS Course FROM assignment WHERE submit_date > due_date AND  grade BETWEEN 60 AND 70 OR course_id != 3
 ```
 
 
 ### Additional Topics, time permitting
 * SSMS (SQL Server Management Studio) for Microsoft SQL Server
 * Azure Data Studio - also from Microsoft, more lightweight, like VS Core can connect to most SQL
 * JOINS (using WHERE to join is .. ok.. using JOIN  ON is better)
 * * table aliases
 * ORDER BY 
 * GROUP BY
 